In [10]:
from dotenv import load_dotenv
from openai import OpenAI

import numpy as np

import json
from concurrent import futures
from copy import deepcopy
from typing import Any, Callable, Dict, List, Optional, TypedDict, Union
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

load_dotenv()


class LLModelConfig(TypedDict):
    model: str
    base_url: Optional[str]
    api_key: Optional[str]


class LLModel:
    GPT4_0125 = LLModelConfig(model="gpt-4-0125-preview", base_url=None, api_key=None)
    GPT3_0125 = LLModelConfig(model="gpt-3.5-turbo-0125", base_url=None, api_key=None)


class OpenAICompatibleChatCompletion:
    clients: Dict = dict()
    LLM_MAX_OUTER_TOOL_CALLS = 10
    LLM_MAX_INNER_TOOL_CALLS = 10

    @classmethod
    def _load_client(cls, base_url: Optional[str] = None, api_key: Optional[str] = None) -> OpenAI:
        """
        By default, will connect directly to OpenAI API. However, there is the option to connect
        to endpoints which provide compatibility for the OpenAI API standard. For example, to
        connect to together.ai.
        """
        client_key = (base_url, api_key)
        if cls.clients.get(client_key) is None:
            cls.clients[client_key] = OpenAI(base_url=base_url, api_key=api_key)
        return cls.clients[client_key]

    @classmethod
    def call(
        cls,
        messages: List[Dict],
        model: str,
        base_url: Optional[str] = None,
        api_key: Optional[str] = None,
        **kwargs: Any,
    ) -> Dict:
        """
        For a LLM call without tool calls the end result should look something like:
        {
            'message': {'content': 'How can I help you today', 'role': 'assistant'},
            'model': 'gpt-3.5-turbo-0125'
        }

        For a LLM call with tool calls the end result should look something like:
        {'message': {'content': None,
          'role': 'assistant',
          'tool_calls': [{'function': {'arguments': '{"arg1": "value1"}', 'name': 'func_name'},
                                       'id': 'id1', 'type': 'function'}]}
         'model': 'gpt-3.5-turbo-0125'
        }

        If the LLM provider raises an Exception then we return a dict with the field 'error'.
        For example:
        {'error': {
                'code': 'context_length_exceeded',
                'status_code': 400,
                 'type': 'invalid_request_error',
                 'message': "This model's maximum context length is 16385 tokens."
                 }
        }
        """
        client = cls._load_client(base_url, api_key)
        try:
            resp = client.chat.completions.create(
                model=model,
                messages=messages,
                stream=False,
                **kwargs,
            )
        except Exception as e:
            # handle particulars for each LLM model/provider.
            # OpenAI Particulars:
            return {
                "error": {
                    "code": e.code,
                    "status_code": e.status_code,
                    "type": e.type,
                    "message": e.body["message"],
                }
            }
        # handle particulars for each LLM model/provider.
        # OpenAI Particulars:
        response_message = resp.choices[0].message
        response_message = response_message.model_dump()
        # OpenAPI deprecated the field function_call but still returns it so remove it here
        response_message.pop("function_call", None)
        if response_message["tool_calls"] is None:
            response_message.pop("tool_calls")
        model = resp.model

        # return consistent format. We can add more keys/values in the future as we need them.
        # For example, token usage and so on.
        return {"message": response_message, "model": model}

    @classmethod
    def create_chat_completions_async(cls, task_args_list: List[Dict], concurrency: int = 10) -> List[Dict]:
        """
        Make a series of calls to chat.completions.create endpoint in parallel and collect back
        the results. Returns a list of dicts with the same format as the output of call()
        :param task_args_list: A list of dictionaries where each dictionary contains the keyword
            arguments required for call method.
        :param concurrency: the max number of workers
        """

        def create_chat_task(
            task_args: Dict,
        ) -> Union[Dict]:
            return cls().call(**task_args)

        with futures.ThreadPoolExecutor(max_workers=concurrency) as executor:
            results = list(executor.map(create_chat_task, task_args_list))
        return results

    @classmethod
    def generate_with_function_calling(
        cls,
        messages: List[Dict],
        tools: List[Dict],
        functions_look_up: Dict[str, Callable[..., dict]],
        model: str,
        base_url: Optional[str] = None,
        api_key: Optional[str] = None,
        **kwargs: Any,
    ) -> Dict:
        """Nested and parallel function calling.
        Parallel means that one single call can return multiple tool calls which can
        be called independently and in parallel. Nested means that we also attempt
        followup calls with additional tools calls if the LLM proposes that.

        {
                'message': {
                    'role': 'assistant',
                    'content': "Final message from the assistant.",
                },
                'new_messages': List of OpenAI compatible chat history dicts with the
                                messages and tool calls. It contains every **new** message
                                generated up to and including the final assistant message.
                'model': LLModel.GPT4_0125['model'],
                'tool_calls_details': A dictionary keyed by tool_id with details about each
                    tool call.
        }

        This code executes the tools/functions with the arguments
        and gets back the data and returns to the LLM. Each tool/function must return
        a dictionary with this minimal format:
        {
            'data': This is the data passed back to the LLM
        }
        The 'data' field will be used for the LLM to interpret the answer of the resulting
        tool call. Other keys/values can be passed in this dictionary and those details
        will be stored in the output dictionary 'tool_calls_details'.
        """
        messages = deepcopy(messages)
        calls = 1
        new_messages = []
        tool_calls_details: Dict[str, dict] = dict()
        error_message = {
            "role": "assistant",
            "content": "Sorry, the question is too complex.",  # TODO: work on copy
        }
        too_complex_result = {
            "message": error_message,
            "new_messages": [error_message],
            "show_your_work": [],
            "model": None,
            "tool_calls_details": tool_calls_details,
        }

        def call_tool(tool: Callable, tool_args: Dict) -> Any:
            return tool(**tool_args)

        while calls <= cls.LLM_MAX_OUTER_TOOL_CALLS:
            resp = cls.call(messages, model, base_url, api_key, tools=tools, **kwargs)
            if "error" in resp:
                return resp
            response_message = resp["message"]
            tool_calls = response_message.get("tool_calls", [])
            if len(tool_calls) > cls.LLM_MAX_INNER_TOOL_CALLS:
                return too_complex_result
            if not tool_calls:
                new_messages.append(response_message)
                return {
                    "message": response_message,
                    "new_messages": new_messages,
                    "model": resp["model"],
                    "tool_calls_details": tool_calls_details,
                }
            messages.append(response_message)
            new_messages.append(response_message)

            tools_args_list = [json.loads(t["function"]["arguments"]) for t in tool_calls]
            tools_callables = [functions_look_up[t["function"]["name"]] for t in tool_calls]
            tasks = [(tools_callables[i], tools_args_list[i]) for i in range(len(tool_calls))]

            with futures.ThreadPoolExecutor(max_workers=cls.LLM_MAX_INNER_TOOL_CALLS) as executor:
                tool_results = list(executor.map(lambda p: call_tool(p[0], p[1]), tasks))

            for tool_call, tool_result in zip(tool_calls, tool_results):
                function_data = tool_result["data"]
                tool_info = {
                    "tool_call_id": tool_call["id"],
                    "role": "tool",
                    "name": tool_call["function"]["name"],
                    "content": str(function_data),
                }
                tool_calls_details[tool_call["id"]] = {"tool_result": tool_result, **tool_call}
                messages.append(tool_info)
                new_messages.append(tool_info)
            calls += 1

        return too_complex_result

    def get_embeddings(self, texts):
        def task(text):
            text = text.replace("\n", " ")
            return self.clients[(None, None)].embeddings.create(input=[text], model="text-embedding-3-small").data[0].embedding

        with futures.ThreadPoolExecutor(max_workers=10) as executor:
            results = list(executor.map(task, texts))
        return np.array(results)

In [11]:
llm = OpenAICompatibleChatCompletion()
llm.call(messages=[{"role": "system", "content": "Talk like a pirate and return JSON"}, {"role": "user", "content": "hey"}], **LLModel.GPT3_0125)

{'message': {'content': 'Ahoy matey! What be ye needin\' from this ol\' sea dog? Arrr! Here be the treasure ye seek:\n\n{\n  "greeting": "Ahoy matey!",\n  "response": "What be ye needin\' from this ol\' sea dog? Arrr!",\n  "treasure": "Here be the treasure ye seek"\n}',
  'role': 'assistant'},
 'model': 'gpt-3.5-turbo-0125'}

In [17]:
print(llm.get_embeddings(["HEY", "BYE"]).shape)

(2, 1536)


In [18]:
ecommerce_metrics = [
    {"name": "Total Revenue", "enum": "TOTAL_REVENUE", "description": "The total amount of money earned from all sales"},
    {"name": "Average Order Value", "enum": "AVG_ORDER_VALUE", "description": "The average amount spent per order"},
    {"name": "Conversion Rate", "enum": "CONVERSION_RATE", "description": "Percentage of visitors who make a purchase"},
    {"name": "Cart Abandonment Rate", "enum": "CART_ABANDONMENT_RATE", "description": "Percentage of users who add items to cart but don't purchase"},
    {
        "name": "Customer Lifetime Value",
        "enum": "CUSTOMER_LIFETIME_VALUE",
        "description": "Predicted total revenue from a customer over their lifetime",
    },
    {"name": "Customer Acquisition Cost", "enum": "CUSTOMER_ACQUISITION_COST", "description": "Average cost to acquire a new customer"},
    {"name": "Return on Ad Spend", "enum": "RETURN_ON_AD_SPEND", "description": "Revenue generated for every dollar spent on advertising"},
    {"name": "Net Profit Margin", "enum": "NET_PROFIT_MARGIN", "description": "Percentage of revenue that becomes profit after all expenses"},
    {"name": "Repeat Purchase Rate", "enum": "REPEAT_PURCHASE_RATE", "description": "Percentage of customers who make more than one purchase"},
    {"name": "Average Time to Purchase", "enum": "AVG_TIME_TO_PURCHASE", "description": "Average time between first visit and purchase"},
    {"name": "Product Return Rate", "enum": "PRODUCT_RETURN_RATE", "description": "Percentage of sold products that are returned"},
    {
        "name": "Customer Retention Rate",
        "enum": "CUSTOMER_RETENTION_RATE",
        "description": "Percentage of customers who remain active over a given period",
    },
    {"name": "Email Click-Through Rate", "enum": "EMAIL_CTR", "description": "Percentage of email recipients who clicked on a link in the email"},
    {"name": "Site Traffic", "enum": "SITE_TRAFFIC", "description": "Total number of visitors to the e-commerce site"},
    {
        "name": "Mobile Traffic Percentage",
        "enum": "MOBILE_TRAFFIC_PERCENTAGE",
        "description": "Percentage of total traffic coming from mobile devices",
    },
    {"name": "Average Session Duration", "enum": "AVG_SESSION_DURATION", "description": "Average time users spend on the site per visit"},
    {"name": "Bounce Rate", "enum": "BOUNCE_RATE", "description": "Percentage of visitors who leave the site after viewing only one page"},
    {"name": "Pages per Session", "enum": "PAGES_PER_SESSION", "description": "Average number of pages viewed during a single session"},
    {"name": "Add to Cart Rate", "enum": "ADD_TO_CART_RATE", "description": "Percentage of sessions where a product is added to the cart"},
    {
        "name": "Checkout Completion Rate",
        "enum": "CHECKOUT_COMPLETION_RATE",
        "description": "Percentage of users who complete checkout after starting it",
    },
    {"name": "Average Order Size", "enum": "AVG_ORDER_SIZE", "description": "Average number of items per order"},
    {"name": "Revenue per Visitor", "enum": "REVENUE_PER_VISITOR", "description": "Average revenue generated per site visitor"},
    {
        "name": "Customer Satisfaction Score",
        "enum": "CUSTOMER_SATISFACTION_SCORE",
        "description": "Average rating given by customers for their shopping experience",
    },
    {"name": "Net Promoter Score", "enum": "NET_PROMOTER_SCORE", "description": "Measure of customer loyalty and likelihood to recommend"},
    {"name": "Average Response Time", "enum": "AVG_RESPONSE_TIME", "description": "Average time taken to respond to customer inquiries"},
    {
        "name": "Social Media Engagement Rate",
        "enum": "SOCIAL_MEDIA_ENGAGEMENT_RATE",
        "description": "Percentage of followers who interact with social media posts",
    },
    {
        "name": "Inventory Turnover Ratio",
        "enum": "INVENTORY_TURNOVER_RATIO",
        "description": "Number of times inventory is sold and replaced over a period",
    },
    {"name": "Out-of-Stock Rate", "enum": "OUT_OF_STOCK_RATE", "description": "Percentage of time products are unavailable due to stock issues"},
    {"name": "Average Shipping Time", "enum": "AVG_SHIPPING_TIME", "description": "Average time taken to ship an order after purchase"},
    {"name": "On-Time Delivery Rate", "enum": "ON_TIME_DELIVERY_RATE", "description": "Percentage of orders delivered within the promised timeframe"},
    {"name": "Average Discount per Order", "enum": "AVG_DISCOUNT_PER_ORDER", "description": "Average amount of discount applied per order"},
    {"name": "Coupon Redemption Rate", "enum": "COUPON_REDEMPTION_RATE", "description": "Percentage of orders using a coupon or promotional code"},
    {"name": "Gross Merchandise Value", "enum": "GROSS_MERCHANDISE_VALUE", "description": "Total value of merchandise sold through the platform"},
    {"name": "Average Margin", "enum": "AVG_MARGIN", "description": "Average profit margin across all products"},
    {"name": "Customer Support Tickets", "enum": "CUSTOMER_SUPPORT_TICKETS", "description": "Number of customer support inquiries received"},
    {
        "name": "First Contact Resolution Rate",
        "enum": "FIRST_CONTACT_RESOLUTION_RATE",
        "description": "Percentage of support issues resolved in the first interaction",
    },
    {"name": "Site Search Usage Rate", "enum": "SITE_SEARCH_USAGE_RATE", "description": "Percentage of visitors who use the site's search function"},
    {"name": "Search to Purchase Rate", "enum": "SEARCH_TO_PURCHASE_RATE", "description": "Conversion rate for visitors who use site search"},
    {
        "name": "Average Days to First Purchase",
        "enum": "AVG_DAYS_TO_FIRST_PURCHASE",
        "description": "Average number of days between account creation and first purchase",
    },
    {
        "name": "Product Page View to Purchase Rate",
        "enum": "PRODUCT_PAGE_VIEW_TO_PURCHASE_RATE",
        "description": "Percentage of product page views that result in a purchase",
    },
    {"name": "Average Revenue per User", "enum": "AVG_REVENUE_PER_USER", "description": "Average revenue generated per registered user"},
    {"name": "Customer Churn Rate", "enum": "CUSTOMER_CHURN_RATE", "description": "Percentage of customers who stop purchasing over a given period"},
    {
        "name": "Average Items per Category",
        "enum": "AVG_ITEMS_PER_CATEGORY",
        "description": "Average number of items listed in each product category",
    },
    {"name": "Category Performance", "enum": "CATEGORY_PERFORMANCE", "description": "Revenue or units sold per product category"},
    {
        "name": "New vs Returning Customer Ratio",
        "enum": "NEW_VS_RETURNING_CUSTOMER_RATIO",
        "description": "Ratio of new customers to returning customers",
    },
    {
        "name": "Average Customer Lifespan",
        "enum": "AVG_CUSTOMER_LIFESPAN",
        "description": "Average duration a customer continues to purchase from the store",
    },
    {"name": "Email Subscriber Growth Rate", "enum": "EMAIL_SUBSCRIBER_GROWTH_RATE", "description": "Rate at which email subscriber list is growing"},
    {"name": "Email Unsubscribe Rate", "enum": "EMAIL_UNSUBSCRIBE_RATE", "description": "Percentage of subscribers who unsubscribe from email list"},
    {
        "name": "Affiliate Sales Percentage",
        "enum": "AFFILIATE_SALES_PERCENTAGE",
        "description": "Percentage of sales generated through affiliate marketing",
    },
    {"name": "Mobile Conversion Rate", "enum": "MOBILE_CONVERSION_RATE", "description": "Conversion rate for visitors on mobile devices"},
    {"name": "Desktop Conversion Rate", "enum": "DESKTOP_CONVERSION_RATE", "description": "Conversion rate for visitors on desktop devices"},
    {
        "name": "Average Pages Before Purchase",
        "enum": "AVG_PAGES_BEFORE_PURCHASE",
        "description": "Average number of pages viewed before making a purchase",
    },
    {"name": "Refund Rate", "enum": "REFUND_RATE", "description": "Percentage of purchases that result in a refund"},
    {"name": "Average Refund Processing Time", "enum": "AVG_REFUND_PROCESSING_TIME", "description": "Average time taken to process a refund request"},
    {"name": "Product Review Rate", "enum": "PRODUCT_REVIEW_RATE", "description": "Percentage of purchases that result in a product review"},
    {"name": "Average Product Rating", "enum": "AVG_PRODUCT_RATING", "description": "Average star rating across all product reviews"},
    {
        "name": "Wishlist to Cart Conversion Rate",
        "enum": "WISHLIST_TO_CART_CONVERSION_RATE",
        "description": "Percentage of wishlisted items that are added to cart",
    },
    {"name": "Average Time on Product Page", "enum": "AVG_TIME_ON_PRODUCT_PAGE", "description": "Average time spent by users on product pages"},
    {"name": "Cross-Sell Success Rate", "enum": "CROSS_SELL_SUCCESS_RATE", "description": "Percentage of purchases that include a cross-sold item"},
    {"name": "Upsell Success Rate", "enum": "UPSELL_SUCCESS_RATE", "description": "Percentage of customers who accept an upsell offer"},
    {
        "name": "Average Customer Service Response Time",
        "enum": "AVG_CUSTOMER_SERVICE_RESPONSE_TIME",
        "description": "Average time taken to respond to customer service inquiries",
    },
    {
        "name": "Live Chat Engagement Rate",
        "enum": "LIVE_CHAT_ENGAGEMENT_RATE",
        "description": "Percentage of visitors who engage with live chat feature",
    },
    {
        "name": "Abandoned Cart Recovery Rate",
        "enum": "ABANDONED_CART_RECOVERY_RATE",
        "description": "Percentage of abandoned carts that are recovered through follow-up",
    },
    {"name": "Average Time to Abandon Cart", "enum": "AVG_TIME_TO_ABANDON_CART", "description": "Average time spent before a cart is abandoned"},
    {
        "name": "Product Return Reason Distribution",
        "enum": "PRODUCT_RETURN_REASON_DISTRIBUTION",
        "description": "Distribution of reasons given for product returns",
    },
    {
        "name": "Percentage of Organic Traffic",
        "enum": "PERCENTAGE_ORGANIC_TRAFFIC",
        "description": "Percentage of total traffic coming from organic search",
    },
    {"name": "Percentage of Paid Traffic", "enum": "PERCENTAGE_PAID_TRAFFIC", "description": "Percentage of total traffic coming from paid sources"},
    {"name": "Average Customer Feedback Score", "enum": "AVG_CUSTOMER_FEEDBACK_SCORE", "description": "Average score from customer feedback surveys"},
    {
        "name": "Percentage of Mobile Purchases",
        "enum": "PERCENTAGE_MOBILE_PURCHASES",
        "description": "Percentage of total purchases made on mobile devices",
    },
    {"name": "Average Session Value", "enum": "AVG_SESSION_VALUE", "description": "Average value generated per session"},
    {
        "name": "New Product Adoption Rate",
        "enum": "NEW_PRODUCT_ADOPTION_RATE",
        "description": "Rate at which new products are purchased after launch",
    },
    {
        "name": "Average Days Inventory Outstanding",
        "enum": "AVG_DAYS_INVENTORY_OUTSTANDING",
        "description": "Average number of days inventory is held before being sold",
    },
    {
        "name": "Percentage of Revenue from Top Products",
        "enum": "PERCENTAGE_REVENUE_TOP_PRODUCTS",
        "description": "Percentage of total revenue generated by top-selling products",
    },
    {
        "name": "Average Customer Query Resolution Time",
        "enum": "AVG_CUSTOMER_QUERY_RESOLUTION_TIME",
        "description": "Average time taken to fully resolve a customer query",
    },
    {
        "name": "Shopping Cart Conversion Rate",
        "enum": "SHOPPING_CART_CONVERSION_RATE",
        "description": "Percentage of carts that result in a completed purchase",
    },
    {"name": "Average Number of Login Attempts", "enum": "AVG_LOGIN_ATTEMPTS", "description": "Average number of attempts users make to log in"},
    {"name": "Password Reset Frequency", "enum": "PASSWORD_RESET_FREQUENCY", "description": "Frequency at which users reset their passwords"},
    {"name": "Average Load Time", "enum": "AVG_LOAD_TIME", "description": "Average time taken for pages to load"},
    {"name": "Mobile App Install Rate", "enum": "MOBILE_APP_INSTALL_RATE", "description": "Rate at which mobile app is installed by users"},
    {"name": "Mobile App Uninstall Rate", "enum": "MOBILE_APP_UNINSTALL_RATE", "description": "Rate at which users uninstall the mobile app"},
    {
        "name": "Average Revenue per Mobile App User",
        "enum": "AVG_REVENUE_PER_MOBILE_APP_USER",
        "description": "Average revenue generated per mobile app user",
    },
    {"name": "Push Notification Open Rate", "enum": "PUSH_NOTIFICATION_OPEN_RATE", "description": "Percentage of push notifications that are opened"},
    {
        "name": "Social Media Referral Conversion Rate",
        "enum": "SOCIAL_MEDIA_REFERRAL_CONVERSION_RATE",
        "description": "Conversion rate for traffic from social media platforms",
    },
    {"name": "Average Time to Reorder", "enum": "AVG_TIME_TO_REORDER", "description": "Average time between repeat purchases of the same product"},
    {
        "name": "Product Recommendation Click-Through Rate",
        "enum": "PRODUCT_RECOMMENDATION_CTR",
        "description": "Percentage of product recommendations that are clicked",
    },
    {"name": "Guest Checkout Rate", "enum": "GUEST_CHECKOUT_RATE", "description": "Percentage of purchases made by guest users"},
    {"name": "Account Creation Rate", "enum": "ACCOUNT_CREATION_RATE", "description": "Rate at which new user accounts are created"},
    {
        "name": "Average Products Viewed per Session",
        "enum": "AVG_PRODUCTS_VIEWED_PER_SESSION",
        "description": "Average number of product pages viewed in a single session",
    },
    {"name": "Product Image Zoom Rate", "enum": "PRODUCT_IMAGE_ZOOM_RATE", "description": "Percentage of product page views where images are zoomed"},
    {
        "name": "Product Video View Rate",
        "enum": "PRODUCT_VIDEO_VIEW_RATE",
        "description": "Percentage of product page views where videos are watched",
    },
    {"name": "Average Video Watch Time", "enum": "AVG_VIDEO_WATCH_TIME", "description": "Average duration of product video views"},
    {"name": "Frequency of Price Changes", "enum": "FREQUENCY_PRICE_CHANGES", "description": "How often product prices are updated"},
    {"name": "Price Elasticity of Demand", "enum": "PRICE_ELASTICITY_DEMAND", "description": "Measure of how demand changes with price changes"},
    {"name": "Average Time to Checkout", "enum": "AVG_TIME_TO_CHECKOUT", "description": "Average time taken to complete the checkout process"},
    {"name": "Form Field Error Rate", "enum": "FORM_FIELD_ERROR_RATE", "description": "Percentage of form submissions with errors"},
    {
        "name": "Payment Gateway Error Rate",
        "enum": "PAYMENT_GATEWAY_ERROR_RATE",
        "description": "Percentage of transactions with payment gateway errors",
    },
    {
        "name": "Average Customer Support Tickets per Order",
        "enum": "AVG_SUPPORT_TICKETS_PER_ORDER",
        "description": "Average number of support tickets generated per order",
    },
    {"name": "Product Question Frequency", "enum": "PRODUCT_QUESTION_FREQUENCY", "description": "Average number of questions asked per product"},
    {"name": "Question Response Time", "enum": "QUESTION_RESPONSE_TIME", "description": "Average time taken to respond to product questions"},
    {
        "name": "User-Generated Content Submission Rate",
        "enum": "UGC_SUBMISSION_RATE",
        "description": "Rate at which users submit reviews, photos, or videos",
    },
    {"name": "Influencer Campaign ROI", "enum": "INFLUENCER_CAMPAIGN_ROI", "description": "Return on investment for influencer marketing campaigns"},
    {
        "name": "Average Order Processing Time",
        "enum": "AVG_ORDER_PROCESSING_TIME",
        "description": "Average time taken to process an order before shipping",
    },
    {
        "name": "Shipping Cost as Percentage of Order Value",
        "enum": "SHIPPING_COST_PERCENTAGE",
        "description": "Average shipping cost as a percentage of total order value",
    },
    {
        "name": "Free Shipping Threshold Effectiveness",
        "enum": "FREE_SHIPPING_THRESHOLD_EFFECTIVENESS",
        "description": "Percentage of orders that qualify for free shipping",
    },
    {
        "name": "Average Additional Spend for Free Shipping",
        "enum": "AVG_ADDITIONAL_SPEND_FREE_SHIPPING",
        "description": "Average amount spent to reach free shipping threshold",
    },
    {"name": "Seasonal Sales Variation", "enum": "SEASONAL_SALES_VARIATION", "description": "Measure of how sales fluctuate across seasons"},
    {
        "name": "Holiday Season Sales Percentage",
        "enum": "HOLIDAY_SEASON_SALES_PERCENTAGE",
        "description": "Percentage of annual sales occurring during holiday season",
    },
    {"name": "Average Time Between Sales", "enum": "AVG_TIME_BETWEEN_SALES", "description": "Average time gap between individual sales"},
    {
        "name": "Product Bundling Effectiveness",
        "enum": "PRODUCT_BUNDLING_EFFECTIVENESS",
        "description": "Measure of how well product bundles perform compared to individual products",
    },
    {"name": "Gift Card Purchase Rate", "enum": "GIFT_CARD_PURCHASE_RATE", "description": "Percentage of orders that include gift card purchases"},
    {"name": "Gift Card Redemption Rate", "enum": "GIFT_CARD_REDEMPTION_RATE", "description": "Percentage of sold gift cards that are redeemed"},
    {"name": "Average Gift Card Value", "enum": "AVG_GIFT_CARD_VALUE", "description": "Average monetary value of gift cards purchased"},
    {
        "name": "Loyalty Program Enrollment Rate",
        "enum": "LOYALTY_PROGRAM_ENROLLMENT_RATE",
        "description": "Percentage of customers enrolled in loyalty program",
    },
    {"name": "Loyalty Point Redemption Rate", "enum": "LOYALTY_POINT_REDEMPTION_RATE", "description": "Rate at which loyalty points are redeemed"},
    {"name": "Average Loyalty Program ROI", "enum": "AVG_LOYALTY_PROGRAM_ROI", "description": "Return on investment for the loyalty program"},
    {
        "name": "Product Comparison Usage Rate",
        "enum": "PRODUCT_COMPARISON_USAGE_RATE",
        "description": "Percentage of sessions where product comparison tool is used",
    },
    {"name": "Wishlist Addition Rate", "enum": "WISHLIST_ADDITION_RATE", "description": "Rate at which products are added to wishlists"},
    {"name": "Wishlist Removal Rate", "enum": "WISHLIST_REMOVAL_RATE", "description": "Rate at which products are removed from wishlists"},
    {"name": "Average Wishlist Size", "enum": "AVG_WISHLIST_SIZE", "description": "Average number of items in user wishlists"},
    {
        "name": "Product Stock Alert Subscription Rate",
        "enum": "PRODUCT_STOCK_ALERT_SUBSCRIPTION_RATE",
        "description": "Rate at which users subscribe to out-of-stock product alerts",
    },
    {
        "name": "Stock Alert Conversion Rate",
        "enum": "STOCK_ALERT_CONVERSION_RATE",
        "description": "Percentage of stock alert subscribers who purchase when item is back in stock",
    },
    {"name": "Average Time to Restock", "enum": "AVG_TIME_TO_RESTOCK", "description": "Average time taken to restock out-of-stock items"},
    {
        "name": "Percentage of Sales from Marketplace vs. Direct",
        "enum": "PERCENTAGE_SALES_MARKETPLACE_VS_DIRECT",
        "description": "Distribution of sales between marketplace and direct channels",
    },
    {"name": "Average Seller Rating", "enum": "AVG_SELLER_RATING", "description": "Average rating given to sellers (for marketplace models)"},
    {"name": "Seller Onboarding Time", "enum": "SELLER_ONBOARDING_TIME", "description": "Average time taken to onboard new sellers"},
    {"name": "Seller Churn Rate", "enum": "SELLER_CHURN_RATE", "description": "Rate at which sellers leave the platform"},
    {"name": "Average Commission Rate", "enum": "AVG_COMMISSION_RATE", "description": "Average commission charged to sellers"},
    {
        "name": "International Sales Percentage",
        "enum": "INTERNATIONAL_SALES_PERCENTAGE",
        "description": "Percentage of sales from international customers",
    },
    {"name": "Currency Conversion Rate", "enum": "CURRENCY_CONVERSION_RATE", "description": "Rate at which users switch to a different currency"},
    {"name": "Language Switch Rate", "enum": "LANGUAGE_SWITCH_RATE", "description": "Rate at which users switch the site language"},
    {
        "name": "Average Translation Quality Score",
        "enum": "AVG_TRANSLATION_QUALITY_SCORE",
        "description": "Average quality score for translated content",
    },
    {"name": "Accessibility Score", "enum": "ACCESSIBILITY_SCORE", "description": "Measure of the site's accessibility for users with disabilities"},
    {"name": "Voice Search Usage Rate", "enum": "VOICE_SEARCH_USAGE_RATE", "description": "Percentage of searches conducted using voice commands"},
    {"name": "Image Search Usage Rate", "enum": "IMAGE_SEARCH_USAGE_RATE", "description": "Percentage of searches conducted using image uploads"},
    {"name": "AR Feature Usage Rate", "enum": "AR_FEATURE_USAGE_RATE", "description": "Rate at which augmented reality features are used"},
    {
        "name": "Virtual Try-On Conversion Rate",
        "enum": "VIRTUAL_TRY_ON_CONVERSION_RATE",
        "description": "Conversion rate for products with virtual try-on feature",
    },
    {"name": "AI Chatbot Usage Rate", "enum": "AI_CHATBOT_USAGE_RATE", "description": "Percentage of customer interactions handled by AI chatbots"},
    {
        "name": "AI Chatbot Resolution Rate",
        "enum": "AI_CHATBOT_RESOLUTION_RATE",
        "description": "Percentage of queries successfully resolved by AI chatbots",
    },
    {
        "name": "Progressive Web App Adoption Rate",
        "enum": "PWA_ADOPTION_RATE",
        "description": "Rate at which users adopt the progressive web app version",
    },
    {"name": "Average Page Load Speed", "enum": "AVG_PAGE_LOAD_SPEED", "description": "Average time taken for pages to fully load"},
    {"name": "Server Response Time", "enum": "SERVER_RESPONSE_TIME", "description": "Average time taken for server to respond to requests"},
    {"name": "API Response Time", "enum": "API_RESPONSE_TIME", "description": "Average response time for API calls"},
    {"name": "CDN Cache Hit Rate", "enum": "CDN_CACHE_HIT_RATE", "description": "Percentage of content served from CDN cache"},
    {"name": "Database Query Performance", "enum": "DATABASE_QUERY_PERFORMANCE", "description": "Average execution time of database queries"},
    {
        "name": "Peak Hour Performance Degradation",
        "enum": "PEAK_HOUR_PERFORMANCE_DEGRADATION",
        "description": "Measure of performance decline during peak traffic hours",
    },
    {"name": "Mobile App Crash Rate", "enum": "MOBILE_APP_CRASH_RATE", "description": "Frequency of mobile app crashes"},
    {
        "name": "Browser Compatibility Score",
        "enum": "BROWSER_COMPATIBILITY_SCORE",
        "description": "Measure of site's compatibility across different browsers",
    },
    {
        "name": "Average API Calls per Session",
        "enum": "AVG_API_CALLS_PER_SESSION",
        "description": "Average number of API calls made during a user session",
    },
    {"name": "Third-Party Script Load Time", "enum": "THIRD_PARTY_SCRIPT_LOAD_TIME", "description": "Average load time for third-party scripts"},
]

In [19]:
def prepare_embedding_input(rec):
    return f'{rec["name"]}\n{rec["enum"]}'


ecommerce_metrics_embeddings = llm.get_embeddings([prepare_embedding_input(rec) for rec in ecommerce_metrics])

In [20]:
brands = [
    {"name": "TechNova", "enum": "TECH_NOVA", "description": "Innovative consumer electronics manufacturer"},
    {"name": "EcoGreen", "enum": "ECO_GREEN", "description": "Sustainable home goods and appliances"},
    {"name": "FitFlex", "enum": "FIT_FLEX", "description": "Athletic wear and fitness equipment"},
    {"name": "Luxe Living", "enum": "LUXE_LIVING", "description": "High-end furniture and home decor"},
    {"name": "GourmetDelight", "enum": "GOURMET_DELIGHT", "description": "Premium food and kitchen products"},
    {"name": "Nike", "enum": "NIKE", "description": "Global sportswear and athletic footwear brand"},
    {"name": "Apple", "enum": "APPLE", "description": "Multinational technology company"},
    {"name": "Amazon", "enum": "AMAZON", "description": "E-commerce and cloud computing giant"},
    {"name": "PetPal", "enum": "PET_PAL", "description": "Pet supplies and accessories"},
    {"name": "BeautyBliss", "enum": "BEAUTY_BLISS", "description": "Cosmetics and skincare products"},
    {"name": "Toyota", "enum": "TOYOTA", "description": "Japanese automotive manufacturer"},
    {"name": "Zara", "enum": "ZARA", "description": "Fast fashion clothing retailer"},
    {"name": "BookWorm", "enum": "BOOK_WORM", "description": "Online bookstore and e-reader devices"},
    {"name": "GreenThumb", "enum": "GREEN_THUMB", "description": "Gardening supplies and outdoor furniture"},
    {"name": "ToyWorld", "enum": "TOY_WORLD", "description": "Children's toys and educational products"},
    {"name": "Samsung", "enum": "SAMSUNG", "description": "South Korean electronics conglomerate"},
    {"name": "Adidas", "enum": "ADIDAS", "description": "German sportswear manufacturer"},
    {"name": "IKEA", "enum": "IKEA", "description": "Swedish-founded furniture retailer"},
    {"name": "MusicMaster", "enum": "MUSIC_MASTER", "description": "Musical instruments and accessories"},
    {"name": "CraftCorner", "enum": "CRAFT_CORNER", "description": "Art supplies and craft materials"},
    {"name": "Microsoft", "enum": "MICROSOFT", "description": "Multinational technology corporation"},
    {"name": "Coca-Cola", "enum": "COCA_COLA", "description": "Global beverage company"},
    {"name": "AutoPro", "enum": "AUTO_PRO", "description": "Automotive parts and accessories"},
    {"name": "HomeSmart", "enum": "HOME_SMART", "description": "Smart home devices and systems"},
    {"name": "FreshFarms", "enum": "FRESH_FARMS", "description": "Organic produce and grocery items"},
    {"name": "Google", "enum": "GOOGLE", "description": "Internet-related services and products"},
    {"name": "Nestle", "enum": "NESTLE", "description": "Multinational food and drink company"},
    {"name": "TravelEase", "enum": "TRAVEL_EASE", "description": "Luggage and travel accessories"},
    {"name": "FitBit", "enum": "FITBIT", "description": "Fitness trackers and smartwatches"},
    {"name": "CozyHome", "enum": "COZY_HOME", "description": "Bedding and home textiles"},
    {"name": "Disney", "enum": "DISNEY", "description": "Entertainment and media conglomerate"},
    {"name": "Sony", "enum": "SONY", "description": "Japanese multinational conglomerate"},
    {"name": "OfficeMax", "enum": "OFFICE_MAX", "description": "Office supplies and furniture"},
    {"name": "SportsPro", "enum": "SPORTS_PRO", "description": "Professional sports equipment"},
    {"name": "BabyBliss", "enum": "BABY_BLISS", "description": "Baby products and accessories"},
    {"name": "McDonald's", "enum": "MCDONALDS", "description": "Global fast food restaurant chain"},
    {"name": "Walmart", "enum": "WALMART", "description": "Multinational retail corporation"},
    {"name": "GamerZone", "enum": "GAMER_ZONE", "description": "Video games and gaming accessories"},
    {"name": "CampGear", "enum": "CAMP_GEAR", "description": "Camping and outdoor equipment"},
    {"name": "JewelGlow", "enum": "JEWEL_GLOW", "description": "Jewelry and accessories"},
    {"name": "Starbucks", "enum": "STARBUCKS", "description": "Multinational chain of coffeehouses"},
    {"name": "Dell", "enum": "DELL", "description": "Computer technology company"},
    {"name": "PetSmart", "enum": "PETSMART", "description": "Pet supplies retailer"},
    {"name": "FitnessFuel", "enum": "FITNESS_FUEL", "description": "Nutritional supplements and protein powders"},
    {"name": "ArtisanCrafts", "enum": "ARTISAN_CRAFTS", "description": "Handmade and artisanal products"},
    {"name": "Unilever", "enum": "UNILEVER", "description": "Consumer goods company"},
    {"name": "BMW", "enum": "BMW", "description": "Luxury vehicle manufacturer"},
    {"name": "PartyPop", "enum": "PARTY_POP", "description": "Party supplies and decorations"},
    {"name": "TechGeek", "enum": "TECH_GEEK", "description": "Gadgets and tech accessories"},
    {"name": "GreenEnergy", "enum": "GREEN_ENERGY", "description": "Renewable energy products"},
    {"name": "Lego", "enum": "LEGO", "description": "Plastic construction toys"},
    {"name": "Gap", "enum": "GAP", "description": "American clothing and accessories retailer"},
    {"name": "HealthyHeart", "enum": "HEALTHY_HEART", "description": "Health monitoring devices"},
    {"name": "CoffeeWorld", "enum": "COFFEE_WORLD", "description": "Coffee makers and accessories"},
    {"name": "DiyDepot", "enum": "DIY_DEPOT", "description": "Home improvement and DIY supplies"},
    {"name": "Procter & Gamble", "enum": "PROCTER_AND_GAMBLE", "description": "Consumer goods corporation"},
    {"name": "H&M", "enum": "H_AND_M", "description": "Swedish multinational clothing company"},
    {"name": "VintageFinds", "enum": "VINTAGE_FINDS", "description": "Antique and vintage items"},
    {"name": "FitKitchen", "enum": "FIT_KITCHEN", "description": "Healthy cooking appliances"},
    {"name": "EcoTravel", "enum": "ECO_TRAVEL", "description": "Eco-friendly travel gear"},
    {"name": "Netflix", "enum": "NETFLIX", "description": "Streaming entertainment service"},
    {"name": "Canon", "enum": "CANON", "description": "Imaging and optical products manufacturer"},
    {"name": "GourmetGrills", "enum": "GOURMET_GRILLS", "description": "High-end grilling equipment"},
    {"name": "SmartStudy", "enum": "SMART_STUDY", "description": "Educational technology products"},
    {"name": "PetLuxe", "enum": "PET_LUXE", "description": "Luxury pet products"},
    {"name": "Visa", "enum": "VISA", "description": "Global payments technology company"},
    {"name": "Philips", "enum": "PHILIPS", "description": "Dutch technology company"},
    {"name": "Urban Outfitters", "enum": "URBAN_OUTFITTERS", "description": "Lifestyle retail company"},
    {"name": "FreshBeauty", "enum": "FRESH_BEAUTY", "description": "Natural and organic beauty products"},
    {"name": "HomeChef", "enum": "HOME_CHEF", "description": "Gourmet meal kits and ingredients"},
    {"name": "PayPal", "enum": "PAYPAL", "description": "Online payments system"},
    {"name": "LG", "enum": "LG", "description": "South Korean electronics company"},
    {"name": "SleepWell", "enum": "SLEEP_WELL", "description": "Sleep aids and bedding products"},
    {"name": "FitFashion", "enum": "FIT_FASHION", "description": "Athleisure and activewear"},
    {"name": "CleanGreen", "enum": "CLEAN_GREEN", "description": "Eco-friendly cleaning products"},
    {"name": "Spotify", "enum": "SPOTIFY", "description": "Digital music streaming service"},
    {"name": "Panasonic", "enum": "PANASONIC", "description": "Japanese electronics corporation"},
    {"name": "BookNook", "enum": "BOOK_NOOK", "description": "Bookstore and reading accessories"},
    {"name": "PetFit", "enum": "PET_FIT", "description": "Pet fitness and health products"},
    {"name": "BeautyBoutique", "enum": "BEAUTY_BOUTIQUE", "description": "High-end beauty and skincare"},
    {"name": "MasterCard", "enum": "MASTERCARD", "description": "Multinational financial services corporation"},
    {"name": "Lenovo", "enum": "LENOVO", "description": "Chinese multinational technology company"},
    {"name": "OutdoorAdventure", "enum": "OUTDOOR_ADVENTURE", "description": "Outdoor and adventure gear"},
    {"name": "SmartHome", "enum": "SMART_HOME", "description": "Home automation products"},
    {"name": "GourmetSnacks", "enum": "GOURMET_SNACKS", "description": "Premium snack foods"},
    {"name": "Adobe", "enum": "ADOBE", "description": "Software company"},
    {"name": "Huawei", "enum": "HUAWEI", "description": "Chinese telecommunications company"},
    {"name": "FreshFace", "enum": "FRESH_FACE", "description": "Skincare and facial products"},
    {"name": "PetPamper", "enum": "PET_PAMPER", "description": "Pet grooming and care products"},
    {"name": "EcoFashion", "enum": "ECO_FASHION", "description": "Sustainable clothing and accessories"},
    {"name": "Twitter", "enum": "TWITTER", "description": "Social networking service"},
    {"name": "Bosch", "enum": "BOSCH", "description": "German multinational engineering company"},
    {"name": "KidSmart", "enum": "KID_SMART", "description": "Educational toys and games"},
    {"name": "FitBites", "enum": "FIT_BITES", "description": "Healthy snacks and meal replacements"},
    {"name": "HomeHarmony", "enum": "HOME_HARMONY", "description": "Home organization and storage solutions"},
    {"name": "LinkedIn", "enum": "LINKEDIN", "description": "Business and employment-oriented social networking service"},
    {"name": "Nikon", "enum": "NIKON", "description": "Japanese multinational optics and imaging products corporation"},
    {"name": "GreenGarden", "enum": "GREEN_GARDEN", "description": "Organic gardening supplies"},
    {"name": "TechTrend", "enum": "TECH_TREND", "description": "Cutting-edge technology accessories"},
    {"name": "FitFuel", "enum": "FIT_FUEL", "description": "Sports nutrition and energy products"},
    {"name": "Pinterest", "enum": "PINTEREST", "description": "Image sharing and social media service"},
    {"name": "Dyson", "enum": "DYSON", "description": "Technology company specializing in household appliances"},
    {"name": "EcoClean", "enum": "ECO_CLEAN", "description": "Environmentally friendly cleaning products"},
    {"name": "PetGourmet", "enum": "PET_GOURMET", "description": "Gourmet pet food and treats"},
    {"name": "BeautyTech", "enum": "BEAUTY_TECH", "description": "High-tech beauty devices"},
    {"name": "Airbnb", "enum": "AIRBNB", "description": "Online marketplace for lodging and tourism experiences"},
    {"name": "GoPro", "enum": "GOPRO", "description": "Action camera and mobile app company"},
    {"name": "GreenMobility", "enum": "GREEN_MOBILITY", "description": "Eco-friendly transportation solutions"},
    {"name": "FitTech", "enum": "FIT_TECH", "description": "Fitness technology and wearables"},
    {"name": "HomeArtisan", "enum": "HOME_ARTISAN", "description": "Handcrafted home decor"},
    {"name": "Uber", "enum": "UBER", "description": "Transportation network company"},
    {"name": "Bose", "enum": "BOSE", "description": "Audio equipment company"},
    {"name": "NaturalNurture", "enum": "NATURAL_NURTURE", "description": "Organic baby products"},
    {"name": "TechPlay", "enum": "TECH_PLAY", "description": "Educational technology toys"},
    {"name": "EcoWear", "enum": "ECO_WEAR", "description": "Sustainable clothing brand"},
    {"name": "Etsy", "enum": "ETSY", "description": "E-commerce website focused on handmade or vintage items"},
    {"name": "Garmin", "enum": "GARMIN", "description": "GPS technology company"},
    {"name": "CulinaryCreations", "enum": "CULINARY_CREATIONS", "description": "Gourmet cooking tools and appliances"},
    {"name": "PetTech", "enum": "PET_TECH", "description": "Pet technology and smart devices"},
    {"name": "BeautyEssentials", "enum": "BEAUTY_ESSENTIALS", "description": "Essential beauty and skincare products"},
    {"name": "Slack", "enum": "SLACK", "description": "Business communication platform"},
    {"name": "Roomba", "enum": "ROOMBA", "description": "Robotic vacuum cleaner brand"},
    {"name": "EcoPackage", "enum": "ECO_PACKAGE", "description": "Sustainable packaging solutions"},
    {"name": "FitMind", "enum": "FIT_MIND", "description": "Mental fitness and wellness products"},
    {"name": "SmartKitchen", "enum": "SMART_KITCHEN", "description": "Connected kitchen appliances"},
    {"name": "Zoom", "enum": "ZOOM", "description": "Video communications company"},
    {"name": "GreenPackaging", "enum": "GREEN_PACKAGING", "description": "Eco-friendly packaging materials"},
    {"name": "TechLearn", "enum": "TECH_LEARN", "description": "Online learning platforms and tools"},
    {"name": "PetComfort", "enum": "PET_COMFORT", "description": "Comfort-focused pet products"},
    {"name": "Shopify", "enum": "SHOPIFY", "description": "E-commerce platform for online stores"},
    {"name": "Ring", "enum": "RING", "description": "Smart home security products"},
    {"name": "EcoTravel", "enum": "ECO_TRAVEL", "description": "Sustainable travel gear and accessories"},
    {"name": "FitSleep", "enum": "FIT_SLEEP", "description": "Sleep tracking and improvement products"},
    {"name": "ArtisanEats", "enum": "ARTISAN_EATS", "description": "Artisanal and specialty foods"},
]

In [22]:
brands_embeddings = llm.get_embeddings([prepare_embedding_input(rec) for rec in brands])

In [23]:
sales_channels = [
    {"name": "Amazon", "enum": "AMAZON", "description": "World's largest e-commerce marketplace"},
    {"name": "eBay", "enum": "EBAY", "description": "Global online marketplace for new and used goods"},
    {"name": "Walmart Marketplace", "enum": "WALMART_MARKETPLACE", "description": "Online marketplace by Walmart"},
    {"name": "Etsy", "enum": "ETSY", "description": "E-commerce website focused on handmade or vintage items"},
    {"name": "Own Website", "enum": "OWN_WEBSITE", "description": "Company's proprietary e-commerce website"},
    {"name": "Physical Stores", "enum": "PHYSICAL_STORES", "description": "Brick-and-mortar retail locations"},
    {"name": "Instagram Shopping", "enum": "INSTAGRAM_SHOPPING", "description": "In-app shopping feature on Instagram"},
    {"name": "Facebook Marketplace", "enum": "FACEBOOK_MARKETPLACE", "description": "Peer-to-peer and business-to-consumer marketplace on Facebook"},
    {"name": "Google Shopping", "enum": "GOOGLE_SHOPPING", "description": "Google's shopping search engine"},
    {"name": "Shopify", "enum": "SHOPIFY", "description": "E-commerce platform for online stores and retail point-of-sale systems"},
    {"name": "Target Plus", "enum": "TARGET_PLUS", "description": "Target's third-party seller program"},
    {"name": "Newegg", "enum": "NEWEGG", "description": "Online retailer of computer hardware and consumer electronics"},
    {"name": "Overstock", "enum": "OVERSTOCK", "description": "Online retailer selling a broad range of products"},
    {"name": "Wayfair", "enum": "WAYFAIR", "description": "E-commerce company that sells furniture and home-goods"},
    {"name": "Wish", "enum": "WISH", "description": "Mobile e-commerce platform"},
]

In [26]:
questions = [
    {
        "question": "What was the Total Revenue for Nike and Adidas on Amazon and eBay from January 1, 2024 to March 31, 2024?",
        "expected_metric": "TOTAL_REVENUE",
        "expected_brands": ["NIKE", "ADIDAS"],
        "expected_sales_channels": ["AMAZON", "EBAY"],
        "current_period_start_date": "2024-01-01",
        "current_period_end_date": "2024-03-31",
    },
    {
        "question": "Calculate the Average Order Value for Apple products sold through their Own Website between April 1, 2024 and May 31, 2024.",
        "expected_metric": "AVG_ORDER_VALUE",
        "expected_brands": ["APPLE"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "What was the Conversion Rate for Samsung on Walmart Marketplace during the first quarter of 2024?",
        "expected_metric": "CONVERSION_RATE",
        "expected_brands": ["SAMSUNG"],
        "expected_sales_channels": ["WALMART_MARKETPLACE"],
        "current_period_start_date": "2024-01-01",
        "current_period_end_date": "2024-03-31",
    },
    {
        "question": "Determine the Cart Abandonment Rate for IKEA on their Own Website and Physical Stores from May 1, 2024 to June 9, 2024.",
        "expected_metric": "CART_ABANDONMENT_RATE",
        "expected_brands": ["IKEA"],
        "expected_sales_channels": ["OWN_WEBSITE", "PHYSICAL_STORES"],
        "current_period_start_date": "2024-05-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "What was the Customer Lifetime Value for FitFlex customers who purchased through Instagram Shopping in the last 30 days?",
        "expected_metric": "CUSTOMER_LIFETIME_VALUE",
        "expected_brands": ["FIT_FLEX"],
        "expected_sales_channels": ["INSTAGRAM_SHOPPING"],
        "current_period_start_date": "2024-05-11",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Calculate the Customer Acquisition Cost for EcoGreen on Etsy and Amazon for the month of April 2024.",
        "expected_metric": "CUSTOMER_ACQUISITION_COST",
        "expected_brands": ["ECO_GREEN"],
        "expected_sales_channels": ["ETSY", "AMAZON"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-04-30",
    },
    {
        "question": "What was the Return on Ad Spend for Luxe Living on Google Shopping from February 15, 2024 to May 15, 2024?",
        "expected_metric": "RETURN_ON_AD_SPEND",
        "expected_brands": ["LUXE_LIVING"],
        "expected_sales_channels": ["GOOGLE_SHOPPING"],
        "current_period_start_date": "2024-02-15",
        "current_period_end_date": "2024-05-15",
    },
    {
        "question": "Determine the Net Profit Margin for GourmetDelight on their Own Website for the second quarter of 2024.",
        "expected_metric": "NET_PROFIT_MARGIN",
        "expected_brands": ["GOURMET_DELIGHT"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "What was the Repeat Purchase Rate for PetPal on Chewy and Amazon from March 1, 2024 to May 31, 2024?",
        "expected_metric": "REPEAT_PURCHASE_RATE",
        "expected_brands": ["PET_PAL"],
        "expected_sales_channels": ["AMAZON"],
        "current_period_start_date": "2024-03-01",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "Calculate the Average Time to Purchase for BeautyBliss products on Sephora's website in May 2024.",
        "expected_metric": "AVG_TIME_TO_PURCHASE",
        "expected_brands": ["BEAUTY_BLISS"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-05-01",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "What was the Product Return Rate for Zara on their Own Website and Physical Stores from April 1, 2024 to June 9, 2024?",
        "expected_metric": "PRODUCT_RETURN_RATE",
        "expected_brands": ["ZARA"],
        "expected_sales_channels": ["OWN_WEBSITE", "PHYSICAL_STORES"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Determine the Customer Retention Rate for BookWorm on Amazon and their Own Website for the first half of 2024.",
        "expected_metric": "CUSTOMER_RETENTION_RATE",
        "expected_brands": ["BOOK_WORM"],
        "expected_sales_channels": ["AMAZON", "OWN_WEBSITE"],
        "current_period_start_date": "2024-01-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "What was the Email Click-Through Rate for GreenThumb's marketing campaigns on their Own Website from May 1, 2024 to June 9, 2024?",
        "expected_metric": "EMAIL_CTR",
        "expected_brands": ["GREEN_THUMB"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-05-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Calculate the Site Traffic for ToyWorld on their Own Website and Walmart Marketplace for the second quarter of 2024.",
        "expected_metric": "SITE_TRAFFIC",
        "expected_brands": ["TOY_WORLD"],
        "expected_sales_channels": ["OWN_WEBSITE", "WALMART_MARKETPLACE"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "What was the Mobile Traffic Percentage for Adidas on their Own Website and Instagram Shopping in May 2024?",
        "expected_metric": "MOBILE_TRAFFIC_PERCENTAGE",
        "expected_brands": ["ADIDAS"],
        "expected_sales_channels": ["OWN_WEBSITE", "INSTAGRAM_SHOPPING"],
        "current_period_start_date": "2024-05-01",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "Determine the Average Session Duration for MusicMaster on their Own Website from April 15, 2024 to May 15, 2024.",
        "expected_metric": "AVG_SESSION_DURATION",
        "expected_brands": ["MUSIC_MASTER"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-04-15",
        "current_period_end_date": "2024-05-15",
    },
    {
        "question": "What was the Bounce Rate for CraftCorner on Etsy and their Own Website for the first five months of 2024?",
        "expected_metric": "BOUNCE_RATE",
        "expected_brands": ["CRAFT_CORNER"],
        "expected_sales_channels": ["ETSY", "OWN_WEBSITE"],
        "current_period_start_date": "2024-01-01",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "Calculate the Pages per Session for Microsoft's online store from March 1, 2024 to May 31, 2024.",
        "expected_metric": "PAGES_PER_SESSION",
        "expected_brands": ["MICROSOFT"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-03-01",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "What was the Add to Cart Rate for AutoPro on Amazon and eBay during April 2024?",
        "expected_metric": "ADD_TO_CART_RATE",
        "expected_brands": ["AUTO_PRO"],
        "expected_sales_channels": ["AMAZON", "EBAY"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-04-30",
    },
    {
        "question": "Determine the Checkout Completion Rate for HomeSmart on their Own Website and Google Shopping from May 1, 2024 to June 9, 2024.",
        "expected_metric": "CHECKOUT_COMPLETION_RATE",
        "expected_brands": ["HOME_SMART"],
        "expected_sales_channels": ["OWN_WEBSITE", "GOOGLE_SHOPPING"],
        "current_period_start_date": "2024-05-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "What was the Average Order Size for FreshFarms on Walmart Marketplace and their Own Website in the last 60 days?",
        "expected_metric": "AVG_ORDER_SIZE",
        "expected_brands": ["FRESH_FARMS"],
        "expected_sales_channels": ["WALMART_MARKETPLACE", "OWN_WEBSITE"],
        "current_period_start_date": "2024-04-11",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Calculate the Revenue per Visitor for Nestle on Amazon and Walmart Marketplace for the second quarter of 2024.",
        "expected_metric": "REVENUE_PER_VISITOR",
        "expected_brands": ["NESTLE"],
        "expected_sales_channels": ["AMAZON", "WALMART_MARKETPLACE"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "What was the Customer Satisfaction Score for TravelEase on their Own Website from January 1, 2024 to May 31, 2024?",
        "expected_metric": "CUSTOMER_SATISFACTION_SCORE",
        "expected_brands": ["TRAVEL_EASE"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-01-01",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "Determine the Net Promoter Score for FitBit on Amazon and their Own Website for the month of May 2024.",
        "expected_metric": "NET_PROMOTER_SCORE",
        "expected_brands": ["FITBIT"],
        "expected_sales_channels": ["AMAZON", "OWN_WEBSITE"],
        "current_period_start_date": "2024-05-01",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "What was the Average Response Time for CozyHome's customer service on their Own Website from April 1, 2024 to June 9, 2024?",
        "expected_metric": "AVG_RESPONSE_TIME",
        "expected_brands": ["COZY_HOME"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Calculate the Social Media Engagement Rate for Disney on Instagram Shopping and Facebook Marketplace for the second quarter of 2024.",
        "expected_metric": "SOCIAL_MEDIA_ENGAGEMENT_RATE",
        "expected_brands": ["DISNEY"],
        "expected_sales_channels": ["INSTAGRAM_SHOPPING", "FACEBOOK_MARKETPLACE"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "What was the Inventory Turnover Ratio for Sony on Amazon and their Own Website from March 1, 2024 to May 31, 2024?",
        "expected_metric": "INVENTORY_TURNOVER_RATIO",
        "expected_brands": ["SONY"],
        "expected_sales_channels": ["AMAZON", "OWN_WEBSITE"],
        "current_period_start_date": "2024-03-01",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "Determine the Out-of-Stock Rate for OfficeMax on Staples.com and their Own Website for the month of May 2024.",
        "expected_metric": "OUT_OF_STOCK_RATE",
        "expected_brands": ["OFFICE_MAX"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-05-01",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "What was the Average Shipping Time for SportsPro orders on Amazon and eBay from April 15, 2024 to May 15, 2024?",
        "expected_metric": "AVG_SHIPPING_TIME",
        "expected_brands": ["SPORTS_PRO"],
        "expected_sales_channels": ["AMAZON", "EBAY"],
        "current_period_start_date": "2024-04-15",
        "current_period_end_date": "2024-05-15",
    },
    {
        "question": "Calculate the On-Time Delivery Rate for BabyBliss on their Own Website and Walmart Marketplace for the second quarter of 2024.",
        "expected_metric": "ON_TIME_DELIVERY_RATE",
        "expected_brands": ["BABY_BLISS"],
        "expected_sales_channels": ["OWN_WEBSITE", "WALMART_MARKETPLACE"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "What was the Average Discount per Order for Walmart on their Own Website and Physical Stores from May 1, 2024 to June 9, 2024?",
        "expected_metric": "AVG_DISCOUNT_PER_ORDER",
        "expected_brands": ["WALMART"],
        "expected_sales_channels": ["OWN_WEBSITE", "PHYSICAL_STORES"],
        "current_period_start_date": "2024-05-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Determine the Coupon Redemption Rate for GamerZone on their Own Website and Amazon for the month of April 2024.",
        "expected_metric": "COUPON_REDEMPTION_RATE",
        "expected_brands": ["GAMER_ZONE"],
        "expected_sales_channels": ["OWN_WEBSITE", "AMAZON"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-04-30",
    },
    {
        "question": "What was the Gross Merchandise Value for CampGear on REI.com and their Own Website from January 1, 2024 to May 31, 2024?",
        "expected_metric": "GROSS_MERCHANDISE_VALUE",
        "expected_brands": ["CAMP_GEAR"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-01-01",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "What was the Total Revenue for Apple on Amazon and their Own Website from January 1, 2024 to March 31, 2024?",
        "expected_metric": "TOTAL_REVENUE",
        "expected_brands": ["APPLE"],
        "expected_sales_channels": ["AMAZON", "OWN_WEBSITE"],
        "current_period_start_date": "2024-01-01",
        "current_period_end_date": "2024-03-31",
    },
    {
        "question": "Calculate the Average Order Value for Nike products on Instagram Shopping and Facebook Marketplace between April 15, 2024 and June 1, 2024.",
        "expected_metric": "AVG_ORDER_VALUE",
        "expected_brands": ["NIKE"],
        "expected_sales_channels": ["INSTAGRAM_SHOPPING", "FACEBOOK_MARKETPLACE"],
        "current_period_start_date": "2024-04-15",
        "current_period_end_date": "2024-06-01",
    },
    {
        "question": "What was the Conversion Rate for Samsung on their Own Website from May 1, 2024 to June 9, 2024?",
        "expected_metric": "CONVERSION_RATE",
        "expected_brands": ["SAMSUNG"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-05-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Determine the Customer Lifetime Value for FitFlex and EcoGreen on Amazon and eBay from February 1, 2024 to April 30, 2024.",
        "expected_metric": "CUSTOMER_LIFETIME_VALUE",
        "expected_brands": ["FIT_FLEX", "ECO_GREEN"],
        "expected_sales_channels": ["AMAZON", "EBAY"],
        "current_period_start_date": "2024-02-01",
        "current_period_end_date": "2024-04-30",
    },
    {
        "question": "What was the Customer Acquisition Cost for Luxe Living on Wayfair and Overstock from March 15, 2024 to May 31, 2024?",
        "expected_metric": "CUSTOMER_ACQUISITION_COST",
        "expected_brands": ["LUXE_LIVING"],
        "expected_sales_channels": ["WAYFAIR", "OVERSTOCK"],
        "current_period_start_date": "2024-03-15",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "Calculate the Return on Ad Spend for GourmetDelight on Google Shopping and Walmart Marketplace between January 1, 2024 and June 9, 2024.",
        "expected_metric": "RETURN_ON_AD_SPEND",
        "expected_brands": ["GOURMET_DELIGHT"],
        "expected_sales_channels": ["GOOGLE_SHOPPING", "WALMART_MARKETPLACE"],
        "current_period_start_date": "2024-01-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "What was the Net Profit Margin for PetPal on Etsy and Shopify from April 1, 2024 to June 9, 2024?",
        "expected_metric": "NET_PROFIT_MARGIN",
        "expected_brands": ["PET_PAL"],
        "expected_sales_channels": ["ETSY", "SHOPIFY"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Determine the Repeat Purchase Rate for BeautyBliss on their Own Website and Physical Stores from February 15, 2024 to May 15, 2024.",
        "expected_metric": "REPEAT_PURCHASE_RATE",
        "expected_brands": ["BEAUTY_BLISS"],
        "expected_sales_channels": ["OWN_WEBSITE", "PHYSICAL_STORES"],
        "current_period_start_date": "2024-02-15",
        "current_period_end_date": "2024-05-15",
    },
    {
        "question": "What was the Average Time to Purchase for Toyota on their Own Website from January 1, 2024 to June 9, 2024?",
        "expected_metric": "AVG_TIME_TO_PURCHASE",
        "expected_brands": ["TOYOTA"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-01-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Calculate the Product Return Rate for Zara on Instagram Shopping and Facebook Marketplace between March 1, 2024 and May 31, 2024.",
        "expected_metric": "PRODUCT_RETURN_RATE",
        "expected_brands": ["ZARA"],
        "expected_sales_channels": ["INSTAGRAM_SHOPPING", "FACEBOOK_MARKETPLACE"],
        "current_period_start_date": "2024-03-01",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "What was the Customer Retention Rate for BookWorm on Amazon and their Own Website from April 1, 2024 to June 9, 2024?",
        "expected_metric": "CUSTOMER_RETENTION_RATE",
        "expected_brands": ["BOOK_WORM"],
        "expected_sales_channels": ["AMAZON", "OWN_WEBSITE"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Determine the Email Click-Through Rate for GreenThumb on their Own Website from February 1, 2024 to April 30, 2024.",
        "expected_metric": "EMAIL_CTR",
        "expected_brands": ["GREEN_THUMB"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-02-01",
        "current_period_end_date": "2024-04-30",
    },
    {
        "question": "What was the Site Traffic for ToyWorld on Google Shopping and Target Plus from May 1, 2024 to June 9, 2024?",
        "expected_metric": "SITE_TRAFFIC",
        "expected_brands": ["TOY_WORLD"],
        "expected_sales_channels": ["GOOGLE_SHOPPING", "TARGET_PLUS"],
        "current_period_start_date": "2024-05-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Calculate the Mobile Traffic Percentage for Adidas on their Own Website and Physical Stores between January 15, 2024 and April 15, 2024.",
        "expected_metric": "MOBILE_TRAFFIC_PERCENTAGE",
        "expected_brands": ["ADIDAS"],
        "expected_sales_channels": ["OWN_WEBSITE", "PHYSICAL_STORES"],
        "current_period_start_date": "2024-01-15",
        "current_period_end_date": "2024-04-15",
    },
    {
        "question": "What was the Average Session Duration for MusicMaster on Newegg and eBay from March 1, 2024 to May 31, 2024?",
        "expected_metric": "AVG_SESSION_DURATION",
        "expected_brands": ["MUSIC_MASTER"],
        "expected_sales_channels": ["NEWEGG", "EBAY"],
        "current_period_start_date": "2024-03-01",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "What was the Total Revenue for Apple on Amazon and their Own Website from January 1, 2024 to March 31, 2024?",
        "expected_metric": "TOTAL_REVENUE",
        "expected_brands": ["APPLE"],
        "expected_sales_channels": ["AMAZON", "OWN_WEBSITE"],
        "current_period_start_date": "2024-01-01",
        "current_period_end_date": "2024-03-31",
    },
    {
        "question": "Calculate the Average Order Value for Nike products on Instagram Shopping and Facebook Marketplace between April 15, 2024 and June 1, 2024.",
        "expected_metric": "AVG_ORDER_VALUE",
        "expected_brands": ["NIKE"],
        "expected_sales_channels": ["INSTAGRAM_SHOPPING", "FACEBOOK_MARKETPLACE"],
        "current_period_start_date": "2024-04-15",
        "current_period_end_date": "2024-06-01",
    },
    {
        "question": "What was the Conversion Rate for Samsung on their Own Website from May 1, 2024 to June 9, 2024?",
        "expected_metric": "CONVERSION_RATE",
        "expected_brands": ["SAMSUNG"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-05-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Determine the Customer Lifetime Value for FitFlex and EcoGreen on Amazon and eBay from February 1, 2024 to April 30, 2024.",
        "expected_metric": "CUSTOMER_LIFETIME_VALUE",
        "expected_brands": ["FIT_FLEX", "ECO_GREEN"],
        "expected_sales_channels": ["AMAZON", "EBAY"],
        "current_period_start_date": "2024-02-01",
        "current_period_end_date": "2024-04-30",
    },
    {
        "question": "What was the Customer Acquisition Cost for Luxe Living on Wayfair and Overstock from March 15, 2024 to May 31, 2024?",
        "expected_metric": "CUSTOMER_ACQUISITION_COST",
        "expected_brands": ["LUXE_LIVING"],
        "expected_sales_channels": ["WAYFAIR", "OVERSTOCK"],
        "current_period_start_date": "2024-03-15",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "Calculate the Return on Ad Spend for GourmetDelight on Google Shopping and Walmart Marketplace between January 1, 2024 and June 9, 2024.",
        "expected_metric": "RETURN_ON_AD_SPEND",
        "expected_brands": ["GOURMET_DELIGHT"],
        "expected_sales_channels": ["GOOGLE_SHOPPING", "WALMART_MARKETPLACE"],
        "current_period_start_date": "2024-01-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "What was the Net Profit Margin for PetPal on Etsy and Shopify from April 1, 2024 to June 9, 2024?",
        "expected_metric": "NET_PROFIT_MARGIN",
        "expected_brands": ["PET_PAL"],
        "expected_sales_channels": ["ETSY", "SHOPIFY"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Determine the Repeat Purchase Rate for BeautyBliss on their Own Website and Physical Stores from February 15, 2024 to May 15, 2024.",
        "expected_metric": "REPEAT_PURCHASE_RATE",
        "expected_brands": ["BEAUTY_BLISS"],
        "expected_sales_channels": ["OWN_WEBSITE", "PHYSICAL_STORES"],
        "current_period_start_date": "2024-02-15",
        "current_period_end_date": "2024-05-15",
    },
    {
        "question": "What was the Average Time to Purchase for Toyota on their Own Website from January 1, 2024 to June 9, 2024?",
        "expected_metric": "AVG_TIME_TO_PURCHASE",
        "expected_brands": ["TOYOTA"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-01-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Calculate the Product Return Rate for Zara on Instagram Shopping and Facebook Marketplace between March 1, 2024 and May 31, 2024.",
        "expected_metric": "PRODUCT_RETURN_RATE",
        "expected_brands": ["ZARA"],
        "expected_sales_channels": ["INSTAGRAM_SHOPPING", "FACEBOOK_MARKETPLACE"],
        "current_period_start_date": "2024-03-01",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "What was the Customer Retention Rate for BookWorm on Amazon and their Own Website from April 1, 2024 to June 9, 2024?",
        "expected_metric": "CUSTOMER_RETENTION_RATE",
        "expected_brands": ["BOOK_WORM"],
        "expected_sales_channels": ["AMAZON", "OWN_WEBSITE"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Determine the Email Click-Through Rate for GreenThumb on their Own Website from February 1, 2024 to April 30, 2024.",
        "expected_metric": "EMAIL_CTR",
        "expected_brands": ["GREEN_THUMB"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-02-01",
        "current_period_end_date": "2024-04-30",
    },
    {
        "question": "What was the Site Traffic for ToyWorld on Google Shopping and Target Plus from May 1, 2024 to June 9, 2024?",
        "expected_metric": "SITE_TRAFFIC",
        "expected_brands": ["TOY_WORLD"],
        "expected_sales_channels": ["GOOGLE_SHOPPING", "TARGET_PLUS"],
        "current_period_start_date": "2024-05-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Calculate the Mobile Traffic Percentage for Adidas on their Own Website and Physical Stores between January 15, 2024 and April 15, 2024.",
        "expected_metric": "MOBILE_TRAFFIC_PERCENTAGE",
        "expected_brands": ["ADIDAS"],
        "expected_sales_channels": ["OWN_WEBSITE", "PHYSICAL_STORES"],
        "current_period_start_date": "2024-01-15",
        "current_period_end_date": "2024-04-15",
    },
    {
        "question": "What was the Average Session Duration for MusicMaster on Newegg and eBay from March 1, 2024 to May 31, 2024?",
        "expected_metric": "AVG_SESSION_DURATION",
        "expected_brands": ["MUSIC_MASTER"],
        "expected_sales_channels": ["NEWEGG", "EBAY"],
        "current_period_start_date": "2024-03-01",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "Calculate the Bounce Rate for CraftCorner on Etsy and their Own Website between February 1, 2024 and April 30, 2024.",
        "expected_metric": "BOUNCE_RATE",
        "expected_brands": ["CRAFT_CORNER"],
        "expected_sales_channels": ["ETSY", "OWN_WEBSITE"],
        "current_period_start_date": "2024-02-01",
        "current_period_end_date": "2024-04-30",
    },
    {
        "question": "What was the Pages per Session for Microsoft products on Amazon and Newegg from April 1, 2024 to June 9, 2024?",
        "expected_metric": "PAGES_PER_SESSION",
        "expected_brands": ["MICROSOFT"],
        "expected_sales_channels": ["AMAZON", "NEWEGG"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Determine the Add to Cart Rate for Coca-Cola on Walmart Marketplace and their Own Website from January 1, 2024 to March 31, 2024.",
        "expected_metric": "ADD_TO_CART_RATE",
        "expected_brands": ["COCA_COLA"],
        "expected_sales_channels": ["WALMART_MARKETPLACE", "OWN_WEBSITE"],
        "current_period_start_date": "2024-01-01",
        "current_period_end_date": "2024-03-31",
    },
    {
        "question": "What was the Checkout Completion Rate for AutoPro on eBay and Amazon from May 1, 2024 to June 9, 2024?",
        "expected_metric": "CHECKOUT_COMPLETION_RATE",
        "expected_brands": ["AUTO_PRO"],
        "expected_sales_channels": ["EBAY", "AMAZON"],
        "current_period_start_date": "2024-05-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Calculate the Average Order Size for HomeSmart on Google Shopping and Instagram Shopping between March 15, 2024 and May 31, 2024.",
        "expected_metric": "AVG_ORDER_SIZE",
        "expected_brands": ["HOME_SMART"],
        "expected_sales_channels": ["GOOGLE_SHOPPING", "INSTAGRAM_SHOPPING"],
        "current_period_start_date": "2024-03-15",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "What was the Revenue per Visitor for FreshFarms on their Own Website and Physical Stores from February 15, 2024 to May 15, 2024?",
        "expected_metric": "REVENUE_PER_VISITOR",
        "expected_brands": ["FRESH_FARMS"],
        "expected_sales_channels": ["OWN_WEBSITE", "PHYSICAL_STORES"],
        "current_period_start_date": "2024-02-15",
        "current_period_end_date": "2024-05-15",
    },
    {
        "question": "Determine the Customer Satisfaction Score for Google products on their Own Website from April 1, 2024 to June 9, 2024.",
        "expected_metric": "CUSTOMER_SATISFACTION_SCORE",
        "expected_brands": ["GOOGLE"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "What was the Net Promoter Score for Nestle on Amazon and Walmart Marketplace from January 1, 2024 to March 31, 2024?",
        "expected_metric": "NET_PROMOTER_SCORE",
        "expected_brands": ["NESTLE"],
        "expected_sales_channels": ["AMAZON", "WALMART_MARKETPLACE"],
        "current_period_start_date": "2024-01-01",
        "current_period_end_date": "2024-03-31",
    },
    {
        "question": "Calculate the Average Response Time for TravelEase customer support on their Own Website between May 1, 2024 and June 9, 2024.",
        "expected_metric": "AVG_RESPONSE_TIME",
        "expected_brands": ["TRAVEL_EASE"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-05-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "What was the Social Media Engagement Rate for FitBit on Facebook Marketplace and Instagram Shopping from March 1, 2024 to May 31, 2024?",
        "expected_metric": "SOCIAL_MEDIA_ENGAGEMENT_RATE",
        "expected_brands": ["FITBIT"],
        "expected_sales_channels": ["FACEBOOK_MARKETPLACE", "INSTAGRAM_SHOPPING"],
        "current_period_start_date": "2024-03-01",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "Determine the Inventory Turnover Ratio for CozyHome on Wayfair and Overstock from February 1, 2024 to April 30, 2024.",
        "expected_metric": "INVENTORY_TURNOVER_RATIO",
        "expected_brands": ["COZY_HOME"],
        "expected_sales_channels": ["WAYFAIR", "OVERSTOCK"],
        "current_period_start_date": "2024-02-01",
        "current_period_end_date": "2024-04-30",
    },
    {
        "question": "What was the Out-of-Stock Rate for Disney merchandise on their Own Website and Physical Stores from April 15, 2024 to June 9, 2024?",
        "expected_metric": "OUT_OF_STOCK_RATE",
        "expected_brands": ["DISNEY"],
        "expected_sales_channels": ["OWN_WEBSITE", "PHYSICAL_STORES"],
        "current_period_start_date": "2024-04-15",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Calculate the Average Shipping Time for Sony products on Amazon and eBay between January 1, 2024 and March 31, 2024.",
        "expected_metric": "AVG_SHIPPING_TIME",
        "expected_brands": ["SONY"],
        "expected_sales_channels": ["AMAZON", "EBAY"],
        "current_period_start_date": "2024-01-01",
        "current_period_end_date": "2024-03-31",
    },
    {
        "question": "What was the On-Time Delivery Rate for OfficeMax on their Own Website from May 1, 2024 to June 9, 2024?",
        "expected_metric": "ON_TIME_DELIVERY_RATE",
        "expected_brands": ["OFFICE_MAX"],
        "expected_sales_channels": ["OWN_WEBSITE"],
        "current_period_start_date": "2024-05-01",
        "current_period_end_date": "2024-06-09",
    },
    {
        "question": "Determine the Average Discount per Order for SportsPro on Google Shopping and Walmart Marketplace from March 15, 2024 to May 31, 2024.",
        "expected_metric": "AVG_DISCOUNT_PER_ORDER",
        "expected_brands": ["SPORTS_PRO"],
        "expected_sales_channels": ["GOOGLE_SHOPPING", "WALMART_MARKETPLACE"],
        "current_period_start_date": "2024-03-15",
        "current_period_end_date": "2024-05-31",
    },
    {
        "question": "What was the Coupon Redemption Rate for BabyBliss on Etsy and their Own Website from February 15, 2024 to May 15, 2024?",
        "expected_metric": "COUPON_REDEMPTION_RATE",
        "expected_brands": ["BABY_BLISS"],
        "expected_sales_channels": ["ETSY", "OWN_WEBSITE"],
        "current_period_start_date": "2024-02-15",
        "current_period_end_date": "2024-05-15",
    },
    {
        "question": "Calculate the Gross Merchandise Value for McDonald's on their Own Website and Physical Stores between April 1, 2024 and June 9, 2024.",
        "expected_metric": "GROSS_MERCHANDISE_VALUE",
        "expected_brands": ["MCDONALDS"],
        "expected_sales_channels": ["OWN_WEBSITE", "PHYSICAL_STORES"],
        "current_period_start_date": "2024-04-01",
        "current_period_end_date": "2024-06-09",
    },
]

In [27]:
get_backend_metric_tool = {
    "type": "function",
    "function": {
        "name": "get_backend_metric",
        "description": """Takes in the user requested metric and 
        uses ML/AI to return the k nearest neighbors for the most likely related backend ENUM metrics.""",
        "parameters": {
            "type": "object",
            "properties": {
                "user_requested_metric": {
                    "type": "string",
                    "description": "The metric requested by the user.",
                },
            },
            "required": ["user_requested_metric"],
        },
    },
}
get_backend_brands_tool = {
    "type": "function",
    "function": {
        "name": "get_backend_brands",
        "description": """Takes in the user requested brand(s) and 
        uses ML/AI to return the k nearest neighbors for the most likely related backend ENUM brands per requested brands.""",
        "parameters": {
            "type": "object",
            "properties": {
                "user_requested_brands": {
                    "type": "array",
                    "items": {
                        "type": "string",
                    },
                    "default": [],
                    "description": "The list of brand(s) requested by the user.",
                },
            },
            "required": ["user_requested_brands"],
        },
    },
}
get_sales_data_tool = {
    "type": "function",
    "function": {
        "name": "get_sales_data",
        "description": """Get the sales data from the backend system.""",
        "parameters": {
            "type": "object",
            "properties": {
                "backend_metric": {
                    "type": "string",
                    "description": "This is the backend metric ENUM.",
                },
                "backend_brands": {
                    "type": "array",
                    "items": {
                        "type": "string",
                    },
                    "default": [],
                    "description": "The list of backend ENUM brands.",
                },
                "sales_channels": {
                    "type": "array",
                    "items": {
                        "type": "string",
                        "enum": [x["enum"] for x in sales_channels],
                    },
                    "default": [],
                    "description": "The list of sales channels.",
                },
                "current_period_start_date": {
                    "type": "string",
                    "description": "The start of the current reporting period.",
                },
                "current_period_end_date": {
                    "type": "string",
                    "description": "The end of the current reporting period.",
                },
            },
            "required": [
                "backend_metric",
                "backend_brands",
                "current_period_start_date",
                "current_period_end_date",
            ],
        },
    },
}
tools = [get_backend_metric_tool, get_backend_brands_tool, get_sales_data_tool]

In [42]:
def find_k_nearest_neighbors(embeddings, input_embedding, k):
    # Calculate distances
    distances = np.linalg.norm(embeddings - input_embedding, axis=1)

    # Get indices of k smallest distances
    nearest_indices = np.argpartition(distances, k)[:k]

    # Sort the k nearest indices by distance
    nearest_indices = nearest_indices[np.argsort(distances[nearest_indices])]

    return nearest_indices

In [43]:
def get_backend_metric(user_requested_metric: str):
    return {"data": [ecommerce_metrics[i] for i in find_k_nearest_neighbors(llm.get_embeddings([user_requested_metric]), ecommerce_metrics_embeddings, 3)]}


def get_backend_brands(user_requested_brands: list[str]):
    data = dict()
    for brand in user_requested_brands:
        data[brand] = [brands[i] for i in find_k_nearest_neighbors(llm.get_embeddings([brand]), brands_embeddings, 3)]
    return {"data": data}


def get_sales_data(*args, **kwargs):
    return {"data": 10}


functions_look_up = {"get_backend_metric": get_backend_metric, "get_backend_brands": get_backend_brands, "get_sales_data": get_sales_data}

In [61]:
get_backend_brands(["shopify", "nike"])

{'data': {'shopify': [{'name': 'Shopify',
    'enum': 'SHOPIFY',
    'description': 'E-commerce platform for online stores'},
   {'name': 'Etsy',
    'enum': 'ETSY',
    'description': 'E-commerce website focused on handmade or vintage items'},
   {'name': 'Amazon',
    'enum': 'AMAZON',
    'description': 'E-commerce and cloud computing giant'}],
  'nike': [{'name': 'Nike',
    'enum': 'NIKE',
    'description': 'Global sportswear and athletic footwear brand'},
   {'name': 'Nikon',
    'enum': 'NIKON',
    'description': 'Japanese multinational optics and imaging products corporation'},
   {'name': 'Adidas',
    'enum': 'ADIDAS',
    'description': 'German sportswear manufacturer'}]}}

In [62]:
system_prompt = """
You will be asked a question by the user about retrieving sales data.
Use the available tools but only call the tools when needed.
If you need further clarification then ask. 

There are hundreds of metrics and hundreds of brands in the backend system.
The user will not know all these metrics and brands or how to refer to them exactly.
You do not know them all of them either, so I have provided some helper tools for you.

In general you will follow the typical flow when answering questions:
1. Extract the user requested metric and the user requested brand(s).
2. 
    
    a) Pass the user requested metric to the the tool get_backend_metric to
    get the list of most likely corresponding backend metric ENUMs. 
    Then choose the most appropriate from this list. 
    
    b) Pass the user requested brand(s) to the the tool get_backend_brands to
    get the list of most likely corresponding backend brand ENUMs. 
    Then choose the most appropriate from this list. 
    
3. Extract any sales channels if mentioned.
4. Pass the relevant arguments into the get_sales_data tool.


Today's date is Monday, June 10, 2024
"""

In [77]:
def eval_llm_resp(question: dict, llm_resp: dict):
    if not llm_resp.get("tool_calls_details"):
        args_predicted = dict()
    else:
        args_predicted = [
            json.loads(x["function"]["arguments"]) for x in llm_resp["tool_calls_details"].values() if x["function"]["name"] == "get_sales_data"
        ][0]
    return {
        "question": question["question"],
        "expected_metric": question["expected_metric"],
        "predicted_metric": args_predicted.get("backend_metric", ""),
        "metric_correct": question["expected_metric"] == args_predicted.get("backend_metric", ""),
        "expected_brands": sorted(question["expected_brands"]),
        "predicted_brands": sorted(args_predicted.get("backend_brands", [])),
        "brands_correct": sorted(question["expected_brands"]) == sorted(args_predicted.get("backend_brands", [])),
        "expected_sales_channels": sorted(question["expected_sales_channels"]),
        "predicted_sales_channels": sorted(args_predicted.get("sales_channels", [])),
        "sales_channels_correct": sorted(question["expected_sales_channels"]) == sorted(args_predicted.get("sales_channels", [])),
        "expected_current_period_start_date": question["current_period_start_date"],
        "predicted_current_period_start_date": args_predicted.get("current_period_start_date", ""),
        "current_period_start_date_correct": question["current_period_start_date"] == args_predicted.get("current_period_start_date", ""),
        "expected_current_period_end_date": question["current_period_end_date"],
        "predicted_current_period_end_date": args_predicted.get("current_period_end_date", ""),
        "current_period_end_date_correct": question["current_period_end_date"] == args_predicted.get("current_period_end_date", ""),
    }


def eval_questions(questions: list[dict]):
    def task(question: dict):
        llm_resp = OpenAICompatibleChatCompletion.generate_with_function_calling(
            messages=[
                {"role": "system", "content": system_prompt},
                {
                    "role": "user",
                    "content": question["question"],
                },
            ],
            tools=tools,
            functions_look_up=functions_look_up,
            **LLModel.GPT3_0125,
        )
        llm_resp.update(eval_llm_resp(question, llm_resp))
        return llm_resp

    eval_res = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(task, question) for question in questions]

        for future in tqdm(as_completed(futures), total=len(questions), desc="Evaluating questions"):
            eval_res.append(future.result())

    return eval_res

In [85]:
import pandas as pd

df = pd.DataFrame(eval_questions(questions[:20]))

Evaluating questions: 100%|██████████| 20/20 [00:21<00:00,  1.07s/it]


In [86]:
df

,message,new_messages,model,tool_calls_details,question,expected_metric,predicted_metric,metric_correct,expected_brands,predicted_brands,brands_correct,expected_sales_channels,predicted_sales_channels,sales_channels_correct,expected_current_period_start_date,predicted_current_period_start_date,current_period_start_date_correct,expected_current_period_end_date,predicted_current_period_end_date,current_period_end_date_correct
0,{'content': 'The Cart Abandonment Rate for IKE...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_lBJPAFaJxHET9uD1WEaVxMNj': {'tool_resul...,Determine the Cart Abandonment Rate for IKEA o...,CART_ABANDONMENT_RATE,CART_ABANDONMENT_RATE,True,[IKEA],[IKEA],True,"[OWN_WEBSITE, PHYSICAL_STORES]",[],False,2024-05-01,2024-05-01,True,2024-06-09,2024-06-09,True
1,{'content': 'The total revenue for Nike and Ad...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_bePU8RM56sf7QYqsSFo6SqYc': {'tool_resul...,What was the Total Revenue for Nike and Adidas...,TOTAL_REVENUE,TOTAL_REVENUE,True,"[ADIDAS, NIKE]","[ADIDAS, NIKE]",True,"[AMAZON, EBAY]","[AMAZON, EBAY]",True,2024-01-01,2024-01-01,True,2024-03-31,2024-03-31,True
2,{'content': 'The Repeat Purchase Rate for PetP...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_OkK0LLtNr7ZSoRGZzb24BL9M': {'tool_resul...,What was the Repeat Purchase Rate for PetPal o...,REPEAT_PURCHASE_RATE,REPEAT_PURCHASE_RATE,True,[PET_PAL],[PET_PAL],True,[AMAZON],[],False,2024-03-01,2024-03-01,True,2024-05-31,2024-05-31,True
3,{'content': 'The Return on Ad Spend for Luxe L...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_uTUPKV03hqrErWpWBfK5y1gD': {'tool_resul...,What was the Return on Ad Spend for Luxe Livin...,RETURN_ON_AD_SPEND,RETURN_ON_AD_SPEND,True,[LUXE_LIVING],[LUXE_LIVING],True,[GOOGLE_SHOPPING],[GOOGLE_SHOPPING],True,2024-02-15,2024-02-15,True,2024-05-15,2024-05-15,True
4,{'content': 'The Average Order Value for Apple...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_M4WlcHHt1ebKUJW1LNcFZqtX': {'tool_resul...,Calculate the Average Order Value for Apple pr...,AVG_ORDER_VALUE,AVG_ORDER_VALUE,True,[APPLE],[APPLE],True,[OWN_WEBSITE],[OWN_WEBSITE],True,2024-04-01,2024-04-01,True,2024-05-31,2024-05-31,True
5,{'content': 'The Customer Lifetime Value for F...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_LgcuAd3pU5yVTY53z5khTLO0': {'tool_resul...,What was the Customer Lifetime Value for FitFl...,CUSTOMER_LIFETIME_VALUE,CUSTOMER_LIFETIME_VALUE,True,[FIT_FLEX],[FIT_FLEX],True,[INSTAGRAM_SHOPPING],[INSTAGRAM_SHOPPING],True,2024-05-11,2024-05-11,True,2024-06-09,2024-06-10,False
6,{'content': 'I have identified that the Custom...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_vnkwI3zn3N4fJiP9k00Pfhrh': {'tool_resul...,Calculate the Customer Acquisition Cost for Ec...,CUSTOMER_ACQUISITION_COST,CUSTOMER_ACQUISITION_COST,True,[ECO_GREEN],[ECO_GREEN],True,"[AMAZON, ETSY]","[AMAZON, ETSY]",True,2024-04-01,2024-04-01,True,2024-04-30,2024-04-30,True
7,{'content': 'The Conversion Rate for Samsung o...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_tQQSzGWcuqTdBkJ8KOpg8cQD': {'tool_resul...,What was the Conversion Rate for Samsung on Wa...,CONVERSION_RATE,CONVERSION_RATE,True,[SAMSUNG],[SAMSUNG],True,[WALMART_MARKETPLACE],[WALMART_MARKETPLACE],True,2024-01-01,2024-01-01,True,2024-03-31,2024-03-31,True
8,{'content': 'The Product Return Rate for Zara ...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_P4KExzYRxsyYWwV0N0wctMFm': {'tool_resul...,What was the Product Return Rate for Zara on t...,PRODUCT_RETURN_RATE,PRODUCT_RETURN_RATE,True,[ZARA],[ZARA],True,"[OWN_WEBSITE, PHYSICAL_STORES]",[],False,2024-04-01,2024-04-01,True,2024-06-09,2024-06-09,True
9,{'content': 'The Mobile Traffic Percentage for...,"[{'content': None, 'role': 'assistant

In [87]:
def calculate_accuracies(df):
    accuracies = {}
    for col in df.columns:
        if col.endswith("_correct"):
            accuracy = df[col].sum() / df.shape[0]
            accuracies[col.replace('correct', 'accuracy')] = f"{accuracy:.2%}"
    return accuracies

accuracies = calculate_accuracies(df)
print(accuracies)


{'metric_accuracy': '100.00%', 'brands_accuracy': '95.00%', 'sales_channels_accuracy': '55.00%', 'current_period_start_date_accuracy': '100.00%', 'current_period_end_date_accuracy': '80.00%'}


In [89]:
df[~df['sales_channels_correct']]

,message,new_messages,model,tool_calls_details,question,expected_metric,predicted_metric,metric_correct,expected_brands,predicted_brands,brands_correct,expected_sales_channels,predicted_sales_channels,sales_channels_correct,expected_current_period_start_date,predicted_current_period_start_date,current_period_start_date_correct,expected_current_period_end_date,predicted_current_period_end_date,current_period_end_date_correct
0,{'content': 'The Cart Abandonment Rate for IKE...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_lBJPAFaJxHET9uD1WEaVxMNj': {'tool_resul...,Determine the Cart Abandonment Rate for IKEA o...,CART_ABANDONMENT_RATE,CART_ABANDONMENT_RATE,True,[IKEA],[IKEA],True,"[OWN_WEBSITE, PHYSICAL_STORES]",[],False,2024-05-01,2024-05-01,True,2024-06-09,2024-06-09,True
2,{'content': 'The Repeat Purchase Rate for PetP...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_OkK0LLtNr7ZSoRGZzb24BL9M': {'tool_resul...,What was the Repeat Purchase Rate for PetPal o...,REPEAT_PURCHASE_RATE,REPEAT_PURCHASE_RATE,True,[PET_PAL],[PET_PAL],True,[AMAZON],[],False,2024-03-01,2024-03-01,True,2024-05-31,2024-05-31,True
8,{'content': 'The Product Return Rate for Zara ...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_P4KExzYRxsyYWwV0N0wctMFm': {'tool_resul...,What was the Product Return Rate for Zara on t...,PRODUCT_RETURN_RATE,PRODUCT_RETURN_RATE,True,[ZARA],[ZARA],True,"[OWN_WEBSITE, PHYSICAL_STORES]",[],False,2024-04-01,2024-04-01,True,2024-06-09,2024-06-09,True
9,{'content': 'The Mobile Traffic Percentage for...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_gQjzm0pYoRwoheEFSVbAkKpG': {'tool_resul...,What was the Mobile Traffic Percentage for Adi...,MOBILE_TRAFFIC_PERCENTAGE,MOBILE_TRAFFIC_PERCENTAGE,True,[ADIDAS],[ADIDAS],True,"[INSTAGRAM_SHOPPING, OWN_WEBSITE]",[OWN_WEBSITE],False,2024-05-01,2024-05-01,True,2024-05-31,2024-05-31,True
10,{'content': 'The Email Click-Through Rate for ...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_PTDtOynZnm2BGNgVVCVweN4e': {'tool_resul...,What was the Email Click-Through Rate for Gree...,EMAIL_CTR,EMAIL_CTR,True,[GREEN_THUMB],[GREEN_THUMB],True,[OWN_WEBSITE],[],False,2024-05-01,2024-05-01,True,2024-06-09,2024-06-09,True
11,{'content': 'The average session duration for ...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_TrgwNeDQxd8RG3qR9hvysyqW': {'tool_resul...,Determine the Average Session Duration for Mus...,AVG_SESSION_DURATION,AVG_SESSION_DURATION,True,[MUSIC_MASTER],[MUSIC_MASTER],True,[OWN_WEBSITE],[],False,2024-04-15,2024-04-15,True,2024-05-15,2024-05-15,True
12,{'content': 'The Pages per Session for Microso...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_gQF98uMRqNAYThTwhXcKG6B0': {'tool_resul...,Calculate the Pages per Session for Microsoft'...,PAGES_PER_SESSION,PAGES_PER_SESSION,True,[MICROSOFT],[MICROSOFT],True,[OWN_WEBSITE],[],False,2024-03-01,2024-03-01,True,2024-05-31,2024-05-31,True
14,{'content': 'The average time to purchase for ...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_1wtkDPB3gM8oF4yDCQ1Vft9K': {'tool_resul...,Calculate the Average Time to Purchase for Bea...,AVG_TIME_TO_PURCHASE,AVG_TIME_TO_PURCHASE,True,[BEAUTY_BLISS],[BEAUTY_BLISS],True,[OWN_WEBSITE],[],False,2024-05-01,2024-05-01,True,2024-05-31,2024-05-31,True
15,{'content': 'The Site Traffic for ToyWorld on ...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_95HCCu6OVJAV4mmnu2OSiUwA': {'tool_resul...,Calculate the Site Traffic for ToyWorld on the...,SITE_TRAFFIC,SITE_TRAFFIC,True,[TOY_WORLD],[TOY_WORLD],True,"[OWN_WEBSITE, WALMART_MARKETPLACE]",[OWN_WEBSITE],False,2024-04-01,2024-04-01,True,2024-06-09,2024-06-30,False


In [ ]:
import anthropic

client = anthropic.Anthropic()
model = "claude-3-5-sonnet-20240620"
message = client.messages.create(
    model=model,
    max_tokens=1000,
    temperature=0,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[{"role": "user", "content": "Why is the ocean salty?"}, {"role": "assistant", "content": "No, as an AI"}],
)
print(message)

m = {"role": "user", "content": "I'm Jeremy"}
r = client.messages.create(messages=[m], model=model, max_tokens=100)
r